# Notebook implementing the stable tree algorithm in Bertsimas et al. (https://arxiv.org/abs/2305.17299)

In [1]:
import sys
import itertools
from pathlib import Path
src_path = Path("../src/dt-distance").resolve()
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.utils import resample
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from dt_distance.data_processor import DataProcessor
from dt_distance.tree_parser import TreeParser
from dt_distance.distance_calculator import DistanceCalculator
from dt_distance.problem_params import ProblemParams

# set seed for reproducibility
np.random.seed(42)

Parameters listed in the paper:

In [2]:
DEPTHS = list(range(3, 13))
MIN_SAMPLES = [3, 5, 10, 30, 50]

## Loading the breast cancer dataset

In [3]:
data_breast_cancer = load_breast_cancer(as_frame=True)
X_full = data_breast_cancer["data"]
y_full = data_breast_cancer["target"]
print("X_full shape: ", X_full.shape)
X_full.head()

X_full shape:  (569, 30)


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [4]:
print("y_full shape: ", y_full.shape)
y_full.head()

y_full shape:  (569,)


0    0
1    0
2    0
3    0
4    0
Name: target, dtype: int64

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size=0.2, random_state=42)

print("X_train shape: {}, X_test shape: {}".format(X_train.shape, X_test.shape))
print("y_train shape: {}, y_test shape: {}".format(y_train.shape, y_test.shape))

X_train shape: (455, 30), X_test shape: (114, 30)
y_train shape: (455,), y_test shape: (114,)


## Step 1: Split Training set into X0 and X1

In [6]:
def random_train_split(X,y):
    N = X.shape[0]
    indices = np.random.permutation(N)
    X0, y0 = X[indices[:N // 2]], y[indices[:N // 2]]
    return X0, y0

X0, y0 = random_train_split(X_train.values, y_train.values)
X0.shape, y0.shape

((227, 30), (227,))

## Step 3: Bootstrap and Train ($T_{0}$) Tree Set

In [7]:
def train_decision_tree(X, y, depth, min_samples_leaf):
    clf = DecisionTreeClassifier(max_depth=depth, min_samples_leaf=min_samples_leaf)
    clf.fit(X, y)
    return clf

def bootstrap_trees(X, y, depths, min_samples, B):
    '''
    Create B bootstrap trees by sampling with replacement from X_0
    '''
    trees = []
    for _ in range(B):
        X_sample, y_sample = resample(X, y, replace= True)
        depth = np.random.choice(depths)
        min_leaf = np.random.choice(min_samples)
        tree = train_decision_tree(X_sample, y_sample, depth, min_leaf)
        trees.append(tree)
    return trees

T0 = bootstrap_trees(X0, y0, DEPTHS, MIN_SAMPLES, 100)
print("Number of trees in T0:", len(T0))

Number of trees in T0: 100


## Step 4: Train Second Tree Collection: $\mathcal{T}$

Train the second tree collection $\mathcal{T}$ using the same method but on the entire training set.

In [8]:
T = bootstrap_trees(X_train.values, y_train.values, DEPTHS, MIN_SAMPLES, 100)
print("Number of trees in T:", len(T))

Number of trees in T: 100


## Step 5.1: Compute Mean distance for each $T \in T$
- For each tree in $\mathcal{T}$, compute `dt-distance` for all $T \in T_{0}$ and average over all B
- Compute AUC score from Test Data to get out-of-sample predictive power
- Return $B$ average distances 
- Intuition for larger set: Say we get new data in the future-> how much do these new trees (entire set)$\mathcal{T}$ deviate from the previosuly smaller set of trees $T_{0}$?
- only structural differences (via path definitions) matter for problem params, so the path_converstion does not care about the dataset, but the bounds on features, quantification of categories, and assigned class labels as a sequence of splits 

In [ ]:
def compute_centroid_trees(trees_ref, trees_target, X, y):
    distances = []
    for i, tree_ref in enumerate(trees_ref):
        d_ref = 0.0
        for tree_target in trees_target:
            distance_calculator = DistanceCalculator(tree_ref, tree_target, X, y)
            d_ref+= distance_calculator.compute_tree_distance()
        d_ref /= len(trees_target)
        distances.append(d_ref)
    return distances

compute_centroid_trees(T0, T, X_train.values, y_train.values)

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

## Step 5.2: Compute out-of-sample Predicitive Performance 
- ROC_AUC score on test-set (our validation set)

In [ ]:
def evaluate_predictive_power(trees, X_holdout, y_holdout):
    auc_scores = []
    for tree in trees:
        y_proba = tree.predict_proba(X_holdout)[:, 1]
        auc = roc_auc_score(y_holdout, y_proba)
        auc_scores.append(auc)
    return auc_scores

## Step 6: Find the Pareto Optimal Set $\mathcal{T}^{*}$ from $\mathcal{T}$
- multi-objective function to find pareto optimal tree set from $\mathcal{T}$ based on average distance, $d_{b}$ , $\forall b \in \mathcal{T}$ and the out-of-sample AUC_ROC score $a_{b}$, $\forall b \in \mathcal{T}$
- **Pareto Optimal Definition:** $(d_{b'} \leq d_b \text{ and } \alpha_{b'} > \alpha_b) \text{ or } (d_{b'} < d_b \text{ and } \alpha_{b'} \geq \alpha_b)$

In [ ]:
def pareto_optimal_trees(distances, auc_scores):
    pareto_trees = []
    for i, (d_i, a_i) in enumerate(zip(distances, auc_scores)):
        dominated = False
        for j, (d_j, a_j) in enumerate(zip(distances, auc_scores)):
            if i != j and ((d_j <= d_i and a_j > a_i) or (d_j < d_i and a_j >= a_i)):
                dominated = True
                break
        if not dominated:
            pareto_trees.append(i)
    return pareto_trees

## Step 7: Find the Optimal Tree from the Pareto Optimal Set, $\mathcal{T^{*}}$
-  $\mathbb{T}^\star = \underset{\mathbb{T}_b \in \mathcal{T}^\star}{\text{argmax}} \ f(d_b, \alpha_b)$
- need to consider here what we value: stability or predicitve power.
-  current function is most stable model among all “good enough” performers.
- Can modify to find optimal trade-off for accuracy-stability
- Indicator function where:
    - 1 if $\alpha_{b}$ is within ε of the best score
    - 0 otherwise

In [ ]:
def select_final_tree(distances, auc_scores, pareto_indices, epsilon=0.01):
    best_auc = max(auc_scores)
    candidates = [i for i in pareto_indices if auc_scores[i] >= (1 - epsilon) * best_auc]
    if not candidates:
        candidates = pareto_indices
    best_idx = max(candidates, key=lambda i: auc_scores[i] - distances[i])
    return best_idx

In [ ]:

# Main method implementing the training of stable trees
def train_stable_tree(X, y, X_holdout, y_holdout, B=20):
    # Parameters
    depths = list(range(3, 13))
    min_samples = [3, 5, 10, 30, 50]


    # Step 2: Train initial collection of trees
    trees_batch_0 = bootstrap_trees(X0, y0, depths, min_samples, B)

    # Step 3: Train second collection of trees on entire data
    trees_full_batch = bootstrap_trees(X, y, depths, min_samples, B)

    # Step 4: Compute stability and predictive performance
    distances = compute_centroid_trees(trees_batch_0, trees_full_batch)
    auc_scores = evaluate_predictive_power(trees_full_batch, X_holdout, y_holdout)

    # Step 5: Pareto frontier
    pareto_indices = pareto_optimal_trees(distances, auc_scores)

    # Step 6: Select optimal stable tree
    best_tree_idx = select_final_tree(distances, auc_scores, pareto_indices)

    #step 7: Choose the best stable tree
    stable_tree = trees_full_batch[best_tree_idx]

    return stable_tree, distances[best_tree_idx], auc_scores[best_tree_idx]